## 4.reflection(google adk)

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from google.adk.agents import LlmAgent, ParallelAgent, SequentialAgent
from google.adk.tools import google_search
GEMINI_MODEL="gemini-2.0-flash"

### 1. 하위 에이전트 정의

In [3]:
# 연구원 1: 재생 에너지
researcher_agent_1 = LlmAgent(
    name="RenewableEnergyResearcher",
    model=GEMINI_MODEL,
    instruction="""
    당신은 에너지 전문 AI 연구 조수입니다.
    '재생 에너지 원천'의 최신 발전을 연구하세요.
    제공된 Google 검색 도구를 사용하세요.
    주요 발견을 간결하게 요약하세요 (1-2 문장).
    요약 결과만 출력하세요.
""",
    description="재생 에너지 원천을 연구합니다.",
    tools=[google_search],
    # 병합 에이전트를 위한 상태에 결과 저장
    output_key="renewable_energy_result"
)

# 연구원 2: 전기 자동차
researcher_agent_2 = LlmAgent(
    name="EVResearcher",
    model=GEMINI_MODEL,
    instruction="""
    당신은 교통 전문 AI 연구 조수입니다.
    '전기 자동차 기술'의 최신 발전을 연구하세요.
    제공된 Google 검색 도구를 사용하세요.
    주요 발견을 간결하게 요약하세요 (1-2 문장).
    요약 결과만 출력하세요.
""",
    description="전기 자동차 기술을 연구합니다.",
    tools=[google_search],
    # 병합 에이전트를 위한 상태에 결과 저장
    output_key="ev_technology_result"
)

# 연구원 3: 탄소 포집
researcher_agent_3 = LlmAgent(
    name="CarbonCaptureResearcher",
    model=GEMINI_MODEL,
    instruction="""
    당신은 기후 솔루션 전문 AI 연구 조수입니다.
    '탄소 포집 방법'의 현재 상태를 연구하세요.
    제공된 Google 검색 도구를 사용하세요.
    주요 발견을 간결하게 요약하세요 (1-2 문장).
    요약 결과만 출력하세요.
""",
    description="탄소 포집 방법을 연구합니다.",
    tools=[google_search],
    # 병합 에이전트를 위한 상태에 결과 저장
    output_key="carbon_capture_result"
)

### 2. ParalleAgent 생성

In [4]:
parallel_research_agent = ParallelAgent(
    name="ParallelWebResearchAgent",
    sub_agents=[researcher_agent_1, researcher_agent_2, researcher_agent_3],
    description="정보를 수집하기 위해 여러 연구 에이전트를 병렬로 실행합니다."
)

### 3. 병합 에이전트 정의

In [5]:
# 이 에이전트는 병렬 에이전트가 세션 상태에 저장한 결과를 가져와 출처를 명시한 단일 구조화된 응답으로 합성합니다

merger_agent = LlmAgent(
    name="SynthesisAgent",
    model=GEMINI_MODEL,  # 또는 합성에 더 강력한 모델이 필요하다면
    instruction="""
    당신은 연구 발견을 구조화된 보고서로 결합하는 AI 조수입니다.
주요 작업은 다음 연구 요약을 합성하여 소스 영역에 명확히 출처를 표시하는 것입니다. 각 주제별 제목을 사용하여 응답을 구조화하세요. 보고서가 일관되고 주요 포인트를 원활하게 통합하도록 하세요.
**중요: 전체 응답은 아래 '입력 요약'에 제공된 정보에 *전적으로* 기반해야 합니다. 이러한 특정 요약에 없는 외부 지식, 사실, 세부 사항을 추가하지 마세요.**
**입력 요약:**
*   **재생 에너지:**
    {renewable_energy_result}
*   **전기 자동차:**
    {ev_technology_result}
*   **탄소 포집:**
    {carbon_capture_result}
**출력 형식:**
## 최근 지속 가능 기술 발전 요약
### 재생 에너지 발견
(RenewableEnergyResearcher의 발견 기반)
[위 재생 에너지 입력 요약에 *만* 기반하여 합성하고 확장하세요.]
### 전기 자동차 발견
(EVResearcher의 발견 기반)
[위 EV 입력 요약에 *만* 기반하여 합성하고 확장하세요.]
### 탄소 포집 발견
(CarbonCaptureResearcher의 발견 기반)
[위 탄소 포집 입력 요약에 *만* 기반하여 합성하고 확장하세요.]
### 전체 결론
[위 발견에 *만* 연결하는 간단한 (1-2 문장) 결론을 제공하세요.]
이 형식을 따르는 구조화된 보고서 *만* 출력하세요. 이 구조 외부에 소개나 결론 문구를 포함하지 말고 제공된 입력 요약 콘텐츠만 엄격히 사용하세요.
""",
    description="병렬 에이전트의 연구 발견을 구조화되고 출처가 명시된 보고서로 결합하며, 제공된 입력에 엄격히 기반합니다.",
    # 병합을 위해 도구 불필요
    # 여기서는 output_key 불필요, 직접 응답이 시퀀스의 최종 출력
)

### 4. Sequential Agent 생성

In [6]:
# 실행될 메인 에이전트입니다. 먼저 ParallelAgent를 실행하여
# 상태를 채운 다음 MergerAgent를 실행하여 최종 출력을 생성합니다
sequential_pipeline_agent = SequentialAgent(
    name="ResearchAndSynthesisPipeline",
    # 먼저 병렬 연구 실행, 그 다음 병합
    sub_agents=[parallel_research_agent, merger_agent],
    description="병렬 연구를 조율하고 결과를 합성합니다."
)

### 5. Sequential Agent 실행 (Async)

작성해볼것